# DataFrames: Operaciones avanzadas
Ahora que ya conocemos las operaciones básicas con *DataFrames* es el momento de ver algunas operaciones más complejas, como por ejemplo:
- Joins
- Agrupaciones complejas
- Operaciones con ventanas
- Rollups
- Cubos
- Pivotes

Para que funcionen algunos ejemplos, esta vez al crear la *SparkSession* usaremos una configuración especial para manejar fechas.

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()
sc = spark.sparkContext

## Joins
Antes de nada vamos a crear unos dataframes de ejemplo para trabajar con joins

In [3]:
person = spark.createDataFrame([(0, "Bill Chambers", 0, [100]), (1, "Matel Zaharla", 1, [500, 250, 100]), (2, "Michael Armbrust", 1, [250, 100])]).toDF("id", "name", "graduate", "marks")
person.show()

graduate_program = spark.createDataFrame([(0, "Masters", "School of Information", "UC Berkeley"), (2, "Masters", "EECS", "UC Berkeley"), (1, "Ph. D.", "EECS", "UC Berkeley")]).toDF("id", "degree", "department", "scool")
graduate_program.show()
spark_status = spark.createDataFrame([(500, "Vice President"), (250, "PMC Member"), (100, "Contributer")]).toDF("id", "status")
spark_status.show()


+---+----------------+--------+---------------+
| id|            name|graduate|          marks|
+---+----------------+--------+---------------+
|  0|   Bill Chambers|       0|          [100]|
|  1|   Matel Zaharla|       1|[500, 250, 100]|
|  2|Michael Armbrust|       1|     [250, 100]|
+---+----------------+--------+---------------+

+---+-------+--------------------+-----------+
| id| degree|          department|      scool|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1| Ph. D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributer|
+---+--------------+



### Inner join
Evalúa las claves en ambos dataframes e incluye sólo las filas cuya evaluación es verdadera

In [5]:
join_expression = person["graduate"] == graduate_program["id"]
person.join(graduate_program,join_expression).show()

+---+----------------+--------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate|          marks| id| degree|          department|      scool|
+---+----------------+--------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|       0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matel Zaharla|       1|[500, 250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|       1|     [250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
+---+----------------+--------+---------------+---+-------+--------------------+-----------+



### Outer join
Evalúa las claves en ambos dataframes y devuelve las filas cuya evaluación es verdadera o falsa, poniendo a NULL los campos para los que no tenga valor

In [6]:
join_type = "outer"
person.join(graduate_program,join_expression, join_type).show()

+----+----------------+--------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate|          marks| id| degree|          department|      scool|
+----+----------------+--------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|       0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matel Zaharla|       1|[500, 250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|       1|     [250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
|NULL|            NULL|    NULL|           NULL|  2|Masters|                EECS|UC Berkeley|
+----+----------------+--------+---------------+---+-------+--------------------+-----------+



### Left Outer join
Evalúa las claves en ambos dataframes y devuelve todas las filas del Dataframe de la izquierda, unidas a las filas del Dataframe de la derecha que han sido evaluadas como verdaderas. Se pone a NUll los camos del Dataframe derecho que no tengan valor

In [7]:
join_type = "left_outer"
person.join(graduate_program,join_expression, join_type).show()

+---+----------------+--------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate|          marks| id| degree|          department|      scool|
+---+----------------+--------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|       0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matel Zaharla|       1|[500, 250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|       1|     [250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
+---+----------------+--------+---------------+---+-------+--------------------+-----------+



### Right outer join
Evalúa las claves en ambos dataframes y devuelve todas las filas del Dataframe de la izquierda, unidas a las filas del Dataframe de la derecha que han sido evaluadas como verdaderas. Se pone a NUll los camos del Dataframe derecho que no tengan valor

In [8]:
join_type = "right_outer"
person.join(graduate_program,join_expression, join_type).show()

+----+----------------+--------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate|          marks| id| degree|          department|      scool|
+----+----------------+--------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|       0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|NULL|            NULL|    NULL|           NULL|  2|Masters|                EECS|UC Berkeley|
|   2|Michael Armbrust|       1|     [250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
|   1|   Matel Zaharla|       1|[500, 250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
+----+----------------+--------+---------------+---+-------+--------------------+-----------+



### Left semi joins
Este tipo de "join" no incluye datos del segundo DataFrame. Simplemente se limita a mostrar las filas del primer DataFrame que tienen sus correspondientes datos en el segundo

In [9]:
join_type = "left_semi"
person.join(graduate_program,join_expression, join_type).show()

+---+----------------+--------+---------------+
| id|            name|graduate|          marks|
+---+----------------+--------+---------------+
|  0|   Bill Chambers|       0|          [100]|
|  1|   Matel Zaharla|       1|[500, 250, 100]|
|  2|Michael Armbrust|       1|     [250, 100]|
+---+----------------+--------+---------------+



### left anti
Sol lo opuesto a los *left semi joins*

In [10]:
join_type = "left_anti"
person.join(graduate_program,join_expression, join_type).show()

+---+----+--------+-----+
| id|name|graduate|marks|
+---+----+--------+-----+
+---+----+--------+-----+



### cross joins
Realiza un producto cartesiano

In [11]:
person.crossJoin(graduate_program).show()

+---+----------------+--------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate|          marks| id| degree|          department|      scool|
+---+----------------+--------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|       0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|   Bill Chambers|       0|          [100]|  2|Masters|                EECS|UC Berkeley|
|  0|   Bill Chambers|       0|          [100]|  1| Ph. D.|                EECS|UC Berkeley|
|  1|   Matel Zaharla|       1|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matel Zaharla|       1|[500, 250, 100]|  2|Masters|                EECS|UC Berkeley|
|  1|   Matel Zaharla|       1|[500, 250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|       1|     [250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  2|Michael Armbrust|       1|     [250, 100]|  2|Masters|           

## Operaciones con ventanas
Se pueden usar *operaciones con ventanas* para conseguir *agregaciones* únicas sobre *"ventanas"* específicas de datos, que se definen referenciando los datos actuales. Esta especificación de ventana define que columnas se pasarán a esta función. Vamos a ver en qué se diferencia de un *group-by* clásico.

En un *group-by*, cada fila pertenece solamente a una agrupación. Una *función de ventana* devuelve un valor para cada fila de entrada basado en un grupo de filas, al que llamaremos *marco*. Cada fila puede pertenecer a más de un marco. Un ejemplo sería calcular la media de un determinado valor para el cual cada fila representa un día de la semana. Así, cada fila acabaría en siete marcos diferentes.
![Funciones con ventanas](./images/windows.png)
Vamos a añadir una nueva columna a nuestro Dataframe que contiene la fecha pero sin horas ni minutos:

In [4]:
from pyspark.sql.functions import col, to_timestamp, date_format
#df_con_timestamp = df_ag.withColumn("timestamp", to_timestamp(col("InvoiceDate"), "MM/d/yyyy H:mm"))
#df_con_fecha = df_con_timestamp.withColumn("date", date_format(col("timestamp"),"yyyy-MM-dd"))

df_ag = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/home/jovyan/work/data/retail-data/all/*.csv")\
  .coalesce(5)
df_ag.cache()
df_ag.createOrReplaceTempView("dfTable")
df_ag.printSchema()

df_con_fecha = df_ag.withColumn("date", date_format(to_timestamp(col("InvoiceDate"), "MM/d/yyyy H:mm"), "yyyy-MM-dd"))
df_con_fecha = df_con_fecha.drop('InvoiceDate')
df_con_fecha.createOrReplaceTempView("df_con_fecha")
df_con_fecha.show()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|     3.39|     17

El primer paso para una *función de ventana* es crear una *especificación de ventana*. La función *partitionBy* no está relacionada con el concepto de particiones visto hasta ahora, simplemente especifica como "partiremos" en grupos. La función *orderBy* determina el orden dentro de cada partición. Por último, la función *rowsBetween* especifica que filas se incluyen en el marco en función de la fila de entrada actual. En el siguiente ejemplo, nos fijamos entodas las filas anteriores a la actual:

In [40]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

windowSpec = Window\
    .partitionBy("CustomerId", "date")\
    .orderBy(desc("Quantity"))\
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

El siguiente paso es usar una *agregación* para, en este caso, aprender más sobre cada cliente (customer). Un ejemplo podría ser obtener la cantidad (quantity) máxima de compra durante todos los tiempos. Para ello vamos a usar una de las funcionesya vistas, indicando la especificación de ventana que define los marcos de datos sobre los que se aplicará esta función.

In [41]:
from pyspark.sql.functions import max

maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)


La función anterior devuelve una columna o expresión. Ésta puede usarse en una sentencia Select de un DataFrame. Antes, es necesario crear el ranking de cantidad (quantity). Para ello podemos usar dos funciones:
- **dense_rank**: Evita la aparición de "huecos" en la secuencia cuando hay valores empatados.
- **rank**: No evita la aparición de "huecos" en la secuencia cuando hay valores empatados.

In [42]:
from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

df_con_fecha.show()

+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|     7.65|     17850|United Kingdom|2010-12-01|
|   536365|    21730|GLASS STAR FROSTE...|       6|     4.25|     17850|United Kingdom|2010-12-01|
|   536366

Esto también devuelve columnas que pueden ser usadas en sentencia *select*. Así podemos construir un *select* para visualizar todos los *valores de ventana* calculados.


In [43]:
df_con_fecha.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
  .select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

## Grouping sets
Hasta ahora hemos visto agrupaciones simples que agregan un conjunto de columnas con los valores presentes en las mismas. Sin embargo, a veces pueden ser necesarias operaciones más complejas, como agregár a través de múltiples grupos. Para ello podemos usar los *grouping sets*. Éstos son una herramienta de bajo nivel que permite combinar conjuntos de agregación.

Para entenderlo mejor, veamos el siguiente ejemplo:

In [31]:
df_no_nulo = df_con_fecha.drop()
df_no_nulo.createOrReplaceTempView("dfNoNulo")
spark.sql("SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNulo GROUP BY customerId, stockCode ORDER BY CustomerId DESC, stockCode DESC").show()

+----------+---------+-------------+
|CustomerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



blablabla

In [32]:
spark.sql("SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNulo GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode)) ORDER BY CustomerId DESC, stockCode DESC").show()

+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



blablabla

In [33]:
spark.sql("SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNulo GROUP BY customerId, stockCode GROUPING SETS((customerId, stockCode),()) ORDER BY CustomerId DESC, stockCode DESC").show()


+----------+---------+-------------+
|customerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



## Rollups
Un *rollup* es una agregación multidimensional que realiza una serie de cálculos *group-by* en la misma operación.

En el siguiente ejemplo se crea un *rollup* que transforma el *DataFrame* de ejemplo en otro que incluye la cantidad total para todas las fechas y países, la cantidad todal por fecha y por país y, por último, la cantidad total para cada fecha.

In [34]:
from pyspark.sql.functions import sum
df_no_nulo.printSchema()
df_enroscado = df_no_nulo.rollup("date", "Country").agg(sum("Quantity")).selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity",).orderBy("Date")
df_enroscado.show()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- date: string (nullable = true)

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      NULL|          NULL|       5176450|
|2010-12-01|          EIRE|           243|
|2010-12-01|     Australia|           107|
|2010-12-01|   Netherlands|            97|
|2010-12-01|       Germany|           117|
|2010-12-01|United Kingdom|         23949|
|2010-12-01|          NULL|         26814|
|2010-12-01|        France|           449|
|2010-12-01|        Norway|          1852|
|2010-12-02|          EIRE|             4|
|2010-12-02|United Kingdom|         20873|
|2010-12-02|       Germany|           146|
|2010-12-02|          NU

Los valores nulos (*null*) representan las cantidades totales. Si ese valor está en ambos campos la cantidad total mostrada se refiere a todas las fechas y todos los países.

In [27]:
df_enroscado.where("Country is Null").show()

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      NULL|   NULL|       5176450|
|2010-12-01|   NULL|         26814|
|2010-12-02|   NULL|         21023|
|2010-12-03|   NULL|         14830|
|2010-12-05|   NULL|         16395|
|2010-12-06|   NULL|         21419|
|2010-12-07|   NULL|         24995|
|2010-12-08|   NULL|         22741|
|2010-12-09|   NULL|         18431|
|2010-12-10|   NULL|         20297|
|2010-12-12|   NULL|         10565|
|2010-12-13|   NULL|         17623|
|2010-12-14|   NULL|         20098|
|2010-12-15|   NULL|         18229|
|2010-12-16|   NULL|         29632|
|2010-12-17|   NULL|         16069|
|2010-12-19|   NULL|          3795|
|2010-12-20|   NULL|         14965|
|2010-12-21|   NULL|         15467|
|2010-12-22|   NULL|          3192|
+----------+-------+--------------+
only showing top 20 rows



In [29]:
df_enroscado.where("date is Null").show()

+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|NULL|   NULL|       5176450|
+----+-------+--------------+



## Cubo
Un *cubo* (*cube*) es un paso má allá respecto al *rollup*. Aquí no se tratan los elementos jerárquicamente si no que se realiza la misma operación a través de todas las dimensiones. Es decir, en el ejemplo, tendríamos los datos agregados totales (nulos en ambos campos), por fecha (nulo en el campo *Country*) y por país (nulo en el campo *date*).

In [36]:
from pyspark.sql.functions import sum
df_cubo = df_no_nulo.cube("date", "Country").agg(sum(col("Quantity"))).select("date", "Country", "sum(Quantity)").orderBy("Date")
df_cubo.show()

+----+--------------+-------------+
|date|       Country|sum(Quantity)|
+----+--------------+-------------+
|NULL|     Australia|        83653|
|NULL|           RSA|          352|
|NULL|        Norway|        19247|
|NULL|       Austria|         4827|
|NULL|          EIRE|       142637|
|NULL|     Lithuania|          652|
|NULL|         Spain|        26824|
|NULL|       Iceland|         2458|
|NULL|       Bahrain|          260|
|NULL|        Greece|         1556|
|NULL|        Israel|         4353|
|NULL|       Finland|        10666|
|NULL|   Switzerland|        30325|
|NULL|         Italy|         7999|
|NULL|     Hong Kong|         4769|
|NULL|United Kingdom|      4263829|
|NULL|   Netherlands|       200128|
|NULL|       Lebanon|          386|
|NULL|      Portugal|        16180|
|NULL|          NULL|      5176450|
+----+--------------+-------------+
only showing top 20 rows



In [37]:
df_cubo.where("date is Null").show()

+----+--------------------+-------------+
|date|             Country|sum(Quantity)|
+----+--------------------+-------------+
|NULL|             Finland|        10666|
|NULL|           Lithuania|          652|
|NULL|              Poland|         3653|
|NULL|             Iceland|         2458|
|NULL|United Arab Emirates|          982|
|NULL|           Singapore|         5234|
|NULL|              Greece|         1556|
|NULL|             Germany|       117448|
|NULL|              France|       110480|
|NULL|               Italy|         7999|
|NULL|               Japan|        25218|
|NULL|      United Kingdom|      4263829|
|NULL|              Sweden|        35637|
|NULL|             Lebanon|          386|
|NULL|             Denmark|         8188|
|NULL|                NULL|      5176450|
|NULL|              Cyprus|         6317|
|NULL|             Austria|         4827|
|NULL|           Australia|        83653|
|NULL|              Norway|        19247|
+----+--------------------+-------

## Pivote
Pivotar permite transformar una fila en una columna. En nuestro *DataFrame* de ejemplo tenemos una columna *Country*. Con un pivote, podemos agregar según una determinada función para cada uno de esos paises y mostrarlosde una forma sencilla.

In [44]:
pivotado = df_con_fecha.groupBy("date").pivot("Country").sum()
pivotado.printSchema()


root
 |-- date: string (nullable = true)
 |-- Australia_sum(Quantity): long (nullable = true)
 |-- Australia_sum(UnitPrice): double (nullable = true)
 |-- Australia_sum(CustomerID): long (nullable = true)
 |-- Austria_sum(Quantity): long (nullable = true)
 |-- Austria_sum(UnitPrice): double (nullable = true)
 |-- Austria_sum(CustomerID): long (nullable = true)
 |-- Bahrain_sum(Quantity): long (nullable = true)
 |-- Bahrain_sum(UnitPrice): double (nullable = true)
 |-- Bahrain_sum(CustomerID): long (nullable = true)
 |-- Belgium_sum(Quantity): long (nullable = true)
 |-- Belgium_sum(UnitPrice): double (nullable = true)
 |-- Belgium_sum(CustomerID): long (nullable = true)
 |-- Brazil_sum(Quantity): long (nullable = true)
 |-- Brazil_sum(UnitPrice): double (nullable = true)
 |-- Brazil_sum(CustomerID): long (nullable = true)
 |-- Canada_sum(Quantity): long (nullable = true)
 |-- Canada_sum(UnitPrice): double (nullable = true)
 |-- Canada_sum(CustomerID): long (nullable = true)
 |-- Channe

Este *DataFrame* tiene una columna para cada combinación de país y un valor numérico.

Ejemplo: USA_sum(Quantity), USA_sum(UnitPrice), etc.

In [45]:
pivotado.where("date > '2011-12-05'").select("date","`USA_sum(Quantity)`").show()

+----------+-----------------+
|      date|USA_sum(Quantity)|
+----------+-----------------+
|2011-12-06|             NULL|
|2011-12-09|             NULL|
|2011-12-08|             -196|
|2011-12-07|             NULL|
+----------+-----------------+

